# TM_Score, GDT_TS, GDT_HA

TM_Score, GDT_TS, GDT_HA scores will be saved as `CASP*/decoys/<target_id>.tmscore.npz`, one per target.

Download and compile the TMscore executable from [Zhang lab](https://zhanglab.ccmb.med.umich.edu/TM-score/):
```bash
wget -q 'https://zhanglab.ccmb.med.umich.edu/TM-score/TMscore.cpp'
g++ -static -O3 -ffast-math -lm -o TMscore TMscore.cpp
rm TMscore.cpp
```

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd

from loguru import logger
from joblib import Parallel, delayed

from graphqa.data.aminoacids import *
from graphqa.data.tmscore import run_tmscore, TmScoreError

Run in parallel on all targets of all CASP editions

In [ ]:
@logger.catch(reraise=False)
def compute_and_save_tm(native_pdb: str, decoys_dir: str, output_npz: str):
    try:
        run_tmscore(native_pdb, decoys_dir, output_npz, tmscore="./TMscore")
    except TmScoreError as e:
        logger.warning(e)


df_natives = pd.read_csv("natives_casp.csv")
with Parallel(n_jobs=30, verbose=1, prefer="threads") as pool:
    missing_targets = [
        dict(
            native_pdb=f"CASP{target.casp_ed}/native/{target.target_id}.pdb",
            decoys_dir=f"CASP{target.casp_ed}/decoys/{target.target_id}",
            output_npz=f"CASP{target.casp_ed}/decoys/{target.target_id}.tmscore.npz",
        )
        for target in df_natives.itertuples()
        if not Path(
            f"CASP{target.casp_ed}/decoys/{target.target_id}.tmscore.npz"
        ).is_file()
    ]
    pool(delayed(compute_and_save_tm)(**target_dict) for target_dict in missing_targets)

Check how many targets failed

In [3]:
pdb = set(p.with_suffix("").name for p in Path().glob("CASP*/native/*.pdb"))
tmscore = set(
    p.with_suffix("").with_suffix("").name
    for p in Path().glob("CASP*/decoys/*.tmscore.npz")
)
failed = pdb - tmscore

if len(failed) > 0:
    logger.warning(f"TMscore failed on {len(failed)} targets")
    if len(failed) < 30:
        for f in failed:
            logger.warning(f"TMscore failed on: {f}")

for p in Path().glob("CASP*/decoys/*.tmscore.npz"):
    count = len(np.load(p)["decoys"])
    if count < 20:
        logger.warning(f"{p} contains {count} decoys")

2020-09-21 23:03:55.748 | WARNING  | __main__:<module>:9 - TMscore failed on 28 targets
2020-09-21 23:03:55.749 | WARNING  | __main__:<module>:12 - TMscore failed on: T0589
2020-09-21 23:03:55.750 | WARNING  | __main__:<module>:12 - TMscore failed on: T0966
2020-09-21 23:03:55.751 | WARNING  | __main__:<module>:12 - TMscore failed on: T0981
2020-09-21 23:03:55.752 | WARNING  | __main__:<module>:12 - TMscore failed on: T0532
2020-09-21 23:03:55.752 | WARNING  | __main__:<module>:12 - TMscore failed on: T0517
2020-09-21 23:03:55.753 | WARNING  | __main__:<module>:12 - TMscore failed on: T0607
2020-09-21 23:03:55.754 | WARNING  | __main__:<module>:12 - TMscore failed on: T0984
2020-09-21 23:03:55.754 | WARNING  | __main__:<module>:12 - TMscore failed on: T0658
2020-09-21 23:03:55.755 | WARNING  | __main__:<module>:12 - TMscore failed on: T0529
2020-09-21 23:03:55.756 | WARNING  | __main__:<module>:12 - TMscore failed on: T0542
2020-09-21 23:03:55.757 | WARNING  | __main__:<module>:12 - TM